In [1]:
#import modules for analyzing ,plotting, and formatting
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import random
import seaborn as sns

In [2]:
original_tf_data = pd.read_csv("first_upload_copy.csv")

df = pd.DataFrame(original_tf_data, columns=['ID Number','Gender','Grade Level','Season','400 Meters','800 Meters','1500 Meters','1600 Meters'])
columnsTitles = ['ID Number','Gender','Grade Level','Season','400 Meters','800 Meters','1500 Meters','1600 Meters']
original_tf_df = df.reindex(columns=columnsTitles)

original_tf_df
copy_df = original_tf_df.copy()

In [3]:
copy_df

,ID Number,Gender,Grade Level,Season,400 Meters,800 Meters,1500 Meters,1600 Meters
0,21,M,12th Grade,2007 Outdoor Season,57.74,NaN,NaN,NaN
1,21,M,12th Grade,2007 Outdoor Season,57.34c,NaN,NaN,NaN
2,21,M,12th Grade,2007 Outdoor Season,56.14c,NaN,NaN,NaN
3,21,M,12th Grade,2007 Outdoor Season,57.2h,NaN,NaN,NaN
4,21,M,12th Grade,2007 Outdoor Season,58.8h,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
41784,19989,M,12th Grade,2005 Outdoor Season,54.14c,NaN,NaN,NaN
41785,19991,M,12th Grade,2005 Outdoor Season,NaN,3:27.3h,NaN,NaN
41786,19991,M,12th Grade,2005 Outdoor Season,NaN,3:14.3h,NaN,NaN
41787,19991,M,12th Grade,2005 Outdoor Season,NaN,3:21.5h,NaN,NaN


In [4]:
copy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41789 entries, 0 to 41788
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID Number    41789 non-null  int64 
 1   Gender       41789 non-null  object
 2   Grade Level  41789 non-null  object
 3   Season       41789 non-null  object
 4   400 Meters   14362 non-null  object
 5   800 Meters   12910 non-null  object
 6   1500 Meters  14315 non-null  object
 7   1600 Meters  200 non-null    object
dtypes: int64(1), object(7)
memory usage: 2.6+ MB


In [5]:
# this is the laters version of these def I am running

null_times = {np.nan,'nan','DNR','nt','no time','FS','DQ','NT','SCR','DNS','DNF','na','MB', 'shot 29.9', 'NS', '?'}

def time_convert(x):
    if (str(x)[-1]=='c') or (str(x)[-1]=='h'):
        x = str(x)[:-1] 
    x_split = str(x).split(':')
    if not '.' in x_split[-1]:
        x = str(x) + '.00'
    #print(x)
    ##Annoying edge cases
    if (x_split[-1] == '60.00') or (x_split[-1] =='60.0') or (x_split[-1] == '28/0'):
      #print('here')
      return np.nan

    if (x[-1] == 'a'):
      #print('here')
      return(np.nan)

    if (str(x) == '57.4**'):
      #print(x)
      return(np.nan)

    if len(x_split)==1:
        t = datetime.datetime.strptime(x, '%S.%f').time()
        return((t.hour * 60 + t.minute) *60 + t.second+ 0.000001*t.microsecond)

    if len(x_split)==2:
      t = datetime.datetime.strptime(x, '%M:%S.%f').time()
      return((t.hour * 60 + t.minute) *60 + t.second+ 0.000001*t.microsecond)

    if len(x_split)==3:
        t = datetime.datetime.strptime(x, '%H:%M:%S.%f').time()
        return((t.hour * 60 + t.minute) *60 + t.second+ 0.000001*t.microsecond)
  

In [6]:
#One dataframe for each event
#here we are doing the final filtering of these df's
#by using a list of wrongly input features, we can check if those values are in any 
#row of these 400m cols. if it is, it is filtered out using the '~'. 
#finally, convert time is applied to convert each input time to seconds. 

df_400 = copy_df[['ID Number', 'Gender', 'Grade Level', 'Season', '400 Meters']]
df_400 = df_400[~df_400['400 Meters'].isin(null_times)]
df_400['400 Meters'] = df_400['400 Meters'].apply(time_convert)

df_800 = copy_df[['ID Number', 'Gender', 'Grade Level', 'Season', '800 Meters']]
df_800 = df_800[~df_800['800 Meters'].isin(null_times)]
df_800['800 Meters'] = df_800['800 Meters'].apply(time_convert)

df_1500 = copy_df[['ID Number', 'Gender', 'Grade Level', 'Season', '1500 Meters']]
df_1500 = df_1500[~df_1500['1500 Meters'].isin(null_times)]
df_1500['1500 Meters'] = df_1500['1500 Meters'].apply(time_convert)

df_1600 = copy_df[['ID Number', 'Gender', 'Grade Level', 'Season', '1600 Meters']]
df_1600 = df_1600[~df_1600['1600 Meters'].isin(null_times)]
df_1600['1600 Meters'] = df_1600['1600 Meters'].apply(time_convert)

In [7]:
df_400.info()
#There is something wrong with these 15 out of 14k in how their times were reported
df_400[(df_400['400 Meters']>180) | (df_400['400 Meters']<40)]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14223 entries, 0 to 41784
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID Number    14223 non-null  int64  
 1   Gender       14223 non-null  object 
 2   Grade Level  14223 non-null  object 
 3   Season       14223 non-null  object 
 4   400 Meters   14188 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 666.7+ KB


,ID Number,Gender,Grade Level,Season,400 Meters
8570,3580,M,11th Grade,2005 Outdoor Season,3304.00
10159,4193,M,12th Grade,2006 Outdoor Season,3213.14
15147,6416,M,12th Grade,2005 Outdoor Season,3694.14
15543,6657,M,9th Grade,2004 Outdoor Season,3573.00
16189,6988,M,9th Grade,2004 Outdoor Season,3375.14
16190,6988,M,9th Grade,2004 Outdoor Season,3429.14
18396,8322,M,10th Grade,2004 Outdoor Season,12.24
18974,8676,M,11th Grade,2005 Outdoor Season,5775.14
26686,12583,M,11th Grade,2007 Outdoor Season,3309.00
27009,12633,M,9th Grade,2005 Outdoor Season,3574.14


In [8]:
df_1500.head()

,ID Number,Gender,Grade Level,Season,1500 Meters
72,63,M,12th Grade,2005 Outdoor Season,291.40
73,63,M,12th Grade,2005 Outdoor Season,267.37
74,63,M,12th Grade,2005 Outdoor Season,267.40
75,63,M,12th Grade,2005 Outdoor Season,261.68
92,63,M,11th Grade,2004 Outdoor Season,265.00


To analyze this dataset, we will be looking at how the runners have changed over time. Specifically we will answer the questions:
1. how has the pop of runners changed over the seasons.
2. how has their performance changed( min, max, mean) by season.
3. how does a 9th grader perform in a recent season vs one in the earliest season.



In [9]:
df_4 = df_400[(df_400['400 Meters']<75) & (df_400['400 Meters']>40)]
df_4.head()

,ID Number,Gender,Grade Level,Season,400 Meters
0,21,M,12th Grade,2007 Outdoor Season,57.74
1,21,M,12th Grade,2007 Outdoor Season,57.34
2,21,M,12th Grade,2007 Outdoor Season,56.14
3,21,M,12th Grade,2007 Outdoor Season,57.20
4,21,M,12th Grade,2007 Outdoor Season,58.80


In [10]:
df_4.shape

(14110, 5)

In [11]:
df_4.to_csv('Runner_Comp_400m.csv', index=False)

In [12]:
# randomly select n rows from your dataset
competitors = df_4.sample(7)

# print the randomly selected rows
print(competitors)

       ID Number Gender Grade Level                                    Season  \
6500        2762      M  12th Grade  2004 Outdoor Season                        
6206        2651      M  11th Grade  2006 Outdoor Season                        
18001       8082      M  11th Grade  2006 Outdoor Season                        
41378      19699      M  10th Grade  2005 Outdoor Season                        
2116        1065      M   9th Grade  2004 Outdoor Season                        
1             21      M  12th Grade  2007 Outdoor Season                        
9112        3850      M  12th Grade  2006 Outdoor Season                        

       400 Meters  
6500        54.64  
6206        61.40  
18001       63.74  
41378       61.04  
2116        54.14  
1           57.34  
9112        56.44  


In [13]:
user_input_1 = input("Enter data for Id Number:")
user_input_2 = input("Enter data for Gender:")
user_input_3 = input("Enter data for Grade Level:")
user_input_4 = input("Enter data for Season:")
user_input_5 = input("Enter data for 400 Meters:")


Enter data for Id Number:100
Enter data for Gender:m
Enter data for Grade Level:12th grade
Enter data for Season:
Enter data for 400 Meters:52


In [14]:
user_data = {"ID Number": [user_input_1], "Gender": [user_input_2], "Grade Level": [user_input_3],\
             "Season": [user_input_4], "400 Meters": [user_input_5]}

user_data

{'ID Number': ['100'],
 'Gender': ['m'],
 'Grade Level': ['12th grade'],
 'Season': [''],
 '400 Meters': ['52']}

In [15]:

# create a new dataframe from the user-generated data
user_data = pd.DataFrame(user_data)

# append the new row to the randomly selected rows
merged_rows = pd.concat([competitors, user_data], ignore_index=True)

# print the randomly selected rows with the new row appended
merged_rows

,ID Number,Gender,Grade Level,Season,400 Meters
0,2762,M,12th Grade,2004 Outdoor Season,54.64
1,2651,M,11th Grade,2006 Outdoor Season,61.4
2,8082,M,11th Grade,2006 Outdoor Season,63.74
3,19699,M,10th Grade,2005 Outdoor Season,61.04
4,1065,M,9th Grade,2004 Outdoor Season,54.14
5,21,M,12th Grade,2007 Outdoor Season,57.34
6,3850,M,12th Grade,2006 Outdoor Season,56.44
7,100,m,12th grade,,52


In [20]:
merged_rows['400 Meters'] = merged_rows['400 Meters'].astype(float)
merged_rows.max()

C:\Users\5luca\AppData\Local\Temp\ipykernel_14716\1639658897.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  merged_rows.max()


Gender                                                m
Grade Level                                   9th Grade
Season         2007 Outdoor Season                     
400 Meters                                        63.74
dtype: object

In [17]:
# find the index of the row with the smallest value in the "400 Meters" column
winner = merged_rows.min()
winner

C:\Users\5luca\AppData\Local\Temp\ipykernel_14716\2739442137.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  winner = merged_rows.min()


Gender                  M
Grade Level    10th Grade
Season                   
400 Meters           52.0
dtype: object

In [18]:
top_3 = merged_rows.nsmallest(3, columns=['400 Meters'])
top_3

,ID Number,Gender,Grade Level,Season,400 Meters
7,100,m,12th grade,,52.00
4,1065,M,9th Grade,2004 Outdoor Season,54.14
0,2762,M,12th Grade,2004 Outdoor Season,54.64
